In [1]:
# Header block to include all modules that must be imported ahead of time
# Only needs to be run once per session, and each time a new module is added

import requests
from bs4 import BeautifulSoup
import json
import pandas as pd # this will need to be installed via command line first
import lxml # this will need to be installed via command line first(as well)

In [13]:
# Method to get the salary cap hits for every player currently signed on an NFL team

def GetTeamSalaryCapHits(team):
    # Use requests to get the raw HTML response
    OTCResponse = requests.get(f'https://overthecap.com/salary-cap/{team[:-4]}')
        
    # If we get a normal response proceed with data scraping
    if OTCResponse.status_code == 200:
        print(f"Request succeeded with status code {OTCResponse.status_code}")
        TeamNameCaptalized = (team[:-4].replace("-", " ")).title()
        print(f"Here are the Salary Cap hits for all players who are currently signed with the {TeamNameCaptalized}:")

        # Pandas read_html method allows the table contents to be put into a DataFrame
        table_tag = 'salary-cap-table contracted-players'
        DataFrames = pd.read_html(OTCResponse.content, attrs={'class': table_tag})
        
        # Select the specific parts of the dataframe we want
        SalariesDF = DataFrames[0].head(51)[['Player', 'Cap Number']]

        # Create a .csv file with all the player names and their cap hits
        SalariesDF.to_csv(f'{TeamNameCaptalized}-Player-Salary-List.csv', index = False, header = False)
        
        # Read the csv back for testing purposes
        SalariesCSV = pd.read_csv(f'{TeamNameCaptalized}-Player-Salary-List.csv', header = None)
        SalariesCSV.columns = ['Player', 'Cap Number']
        print(SalariesCSV)
        

    # If we don't get a normal reponse, stop scraping
    else:
        print(f"Request failed with status code {OTCResponse.status_code}")
        return -1


In [14]:
# A list of links to Over The Cap Salary Cap pages for various teams
NFLTeams = ['buffalo-bills BUF', 'miami-dolphins MIA', 'new-england-patriots NE0', 'new-york-jets NYJ',
            'baltimore-ravens BAL', 'cincinnati-bengals CIN', 'cleveland-browns CLE', 'pittsburgh-steelers PIT',
            'houston-texans HOU', 'indianapolis-colts IND', 'jacksonville-jaguars JAX', 'tennessee-titans TEN', 
            'denver-broncos DEN', 'kansas-city-chiefs KC0', 'las-vegas-raiders LV0', 'los-angeles-chargers LAC',
            'dallas-cowboys DAL', 'new-york-giants NYG', 'philadelphia-eagles PHI', 'washington-commanders WSH',
            'chicago-bears CHI', 'detroit-lions DET', 'green-bay-packers GB0', 'minnesota-vikings MIN',
            'atlanta-falcons ATL', 'carolina-panthers CAR', 'new-orleans-saints NO0', 'tampa-bay-buccaneers TB0',
            'arizona-cardinals ARI', 'los-angeles-rams LAR', 'san-francisco-49ers SF0', 'seattle-seahawks SEA'
           ]

# Call the method defined above to get all the player data
def GetMultipleTeamSalaryCapHits(TeamList):
    for team in TeamList:
        GetTeamSalaryCapHits(team)

# GetMultipleTeamSalaryCapHits(NFLTeams)
GetTeamSalaryCapHits('buffalo-bills BUF')

Request succeeded with status code 200
Here are the Salary Cap hits for all players who are currently signed with the Buffalo Bills:
               Player   Cap Number
0          Josh Allen  $18,636,281
1        Stefon Diggs  $14,875,111
2         Mitch Morse  $11,360,000
3        Dion Dawkins  $10,966,794
4          Micah Hyde  $10,571,294
5          Von Miller   $7,939,000
6       Taron Johnson   $7,037,500
7           Ed Oliver   $4,978,000
8        Jordan Poyer   $4,880,000
9          Tim Settle   $4,324,000
10    Connor McGovern   $4,000,000
11       Deonte Harty   $3,984,500
12         Siran Neal   $3,313,333
13   Gregory Rousseau   $3,165,870
14      Gabriel Davis   $2,917,843
15    Jordan Phillips   $2,860,000
16      Leonard Floyd   $2,623,750
17         Ryan Bates   $2,619,000
18     Dalton Kincaid   $2,441,280
19         Poona Ford   $2,250,000
20         Tyler Bass   $2,076,056
21     Reggie Gilliam   $2,031,333
22       Dane Jackson   $2,010,000
23       Tyrel Dodson   $2,